In [13]:
import networkx as nx
import numpy as np
import pandas as pd

from shapely.geometry import shape
import fiona
import itertools

# Source: https://github.com/eamonustc/geodistance
import geodistance

In [14]:
# Build graph with lat/lon of most visited stuff


def build_graph(shapefile_path,save_graph,num_lanes,max_density,max_speed):

    # Load road network
    # Based on code from:
    # https://gis.stackexchange.com/questions/256955/how-to-load-a-weighed-shapefile-in-networkx
    #X=nx.read_shp(shapefile_path).to_undirected()
    geoms =[shape(feature['geometry']) for feature in fiona.open(shapefile_path)]
    X = nx.Graph()
    for line in geoms:
        for seg_start, seg_end in zip(list(line.coords),list(line.coords)[1:]):
            X.add_edge(seg_start, seg_end) 

    coords = {}
    pos = {k: v for k,v in enumerate(X.nodes())}
    G=nx.Graph()
    for nodenum,coord_pair in pos.items():
        lat = coord_pair[1]
        lon = coord_pair[0]
        G.add_node("intersection_"+str(nodenum), lng=lon, lat=lat, node_type='road_intersection')
        coords["intersection_" + str(nodenum)] = (lat,lon)


    l=[set(x) for x in X.edges()] #To speed things up in case of large objects
    edg=[tuple(k for k,v in pos.items() if v in sl) for sl in l] #Map the G.edges start and endpoints onto pos

    for i in range(0,len(edg)):
        tup = edg[i]
        x = tup[0]
        y = tup[1]
        x = "intersection_" + str(x)
        y = "intersection_" + str(y)
        new_tup = (x,y)
        edg[i] = new_tup

    j = 0
    for edge in edg:
        node1 = edge[0]
        node2 = edge[1]
        data1 = G.nodes[node1]
        data2 = G.nodes[node2]
        dist,br = geodistance.distanceHaversine(data1['lat'],data1['lng'],data2['lat'],data2['lng']) # Distance in km
        fee1 = np.random.uniform(low=1,high=10) # Initialize fees randomly
        G.add_edge(node1,
                   node2,
                   distance=dist,
                   fee=fee1,
                   total_weight=dist+fee1,
                   num_lanes=num_lanes,
                   key="road_"+str(j),
                   occupancy=0,
                   travel_time=0,
                   max_velocity=max_speed,
                   max_density=max_density
                  )
        j += 1



    # Add nodes for venues

    for idex,row in data_ven_new.iterrows():
        node_id = row[0] # We'll need this to identify the nodes
        node_lat = float(row[2])
        node_lon = float(row[3])
        G.add_node('venue_'+str(node_id), lng=node_lon, lat=node_lat, key=node_id, node_type='venue') # can't export if pos is defined; do manually in gephi
        # Connect venue to nearest neighbor

        lowest_nodenum = -1
        lowest_dist = float('inf')
        for nodenum,data in coords.items():
            target_lat = data[0]
            target_lon = data[1]
            dist,br = geodistance.distanceHaversine(node_lat,node_lon,target_lat,target_lon)
            if(dist < lowest_dist):
                lowest_nodenum = nodenum
                lowest_dist = dist
        fee1 = np.random.uniform(low=1,high=10)
        G.add_edge("venue_"+str(node_id),
                   lowest_nodenum,
                   distance=lowest_dist,
                   fee=fee1,
                   total_weight=dist+fee1,
                   num_lanes=num_lanes,
                   key="road_"+str(j),
                   occupancy=0,
                   travel_time=0,
                   max_velocity=max_speed,
                   max_density=999999999999
                  )
        j += 1



    # Export for gephi; display there using geo-layout
    if(save_graph):
        nx.write_gexf(G, "graph_shapefile_taipeigen.gexf")
    return G

In [15]:
shapefile_path = "china-smallest/china-smallest.shp"
num_lanes = 2*2
max_density = 300
max_speed = 100

In [16]:
G = build_graph(shapefile_path,False,num_lanes,max_density,max_speed)

NameError: name 'data_ven_new' is not defined